# Preparación del Escenario: El Reporte del Gerente

In [ ]:
from operator import index

import pandas as pd
import numpy as np

# Reporte "Ancho" (Típico Excel que hace sangrar los ojos de un analista)
data_finanzas = {
    'sucursal': ['Bogota', 'Medellin', 'Cali', 'Bogota', 'Medellin'],
    'categoria': ['Laptops', 'Laptops', 'Laptops', 'Accesorios', 'Accesorios'],
    'Ene-24': [5000, 3000, 2000, 1000, 800],
    'Feb-24': [5200, 3100, 1900, 1100, 850],
    'Mar-24': [4800, 3200, 2100, 1050, 900]
}

df_ancho = pd.DataFrame(data_finanzas)

print("--- REPORTE ORIGINAL (FORMATO ANCHO) ---")
print(df_ancho)

# Ejemplo 1: melt() - Aplanando la curva (De Ancho a Largo)

El Problema: Quieres hacer un gráfico de líneas en Python para ver la tendencia de ventas a lo largo del tiempo. Python te pide una columna que se llame Mes (para el eje X) y otra Ventas (para el eje Y). ¡Pero no las tienes! Tienes los meses regados en los títulos de las columnas.

*La Solución*: Derretir (melt) las columnas de los meses hacia abajo.

In [ ]:
# 'id_vars' son las anclas (Lo que NO se derrite)
# 'value_vars' son las columnas que vamos a aplastar hacia abajo

df_largo = df_ancho.melt(
    id_vars = ['sucursal','categoria'],
    value_vars =['Ene-24','Feb-24','Mar-24'],
    var_name = 'mes', # Nombre para la nueva columna de etiquetas
    value_name = 'ventas' # Nombre para la nueva columna de números
)

print("---- Reporte Estruturado (Formato Largo) ---")
#Se imprime las primeras 10 filas para ver la magia
print(df_largo.head(10))

¿Qué pasó aquí? Las columnas de meses desaparecieron. Ahora tienes una hermosa tabla vertical ideal para bases de datos. La fila de "Laptops en Bogotá" que antes era 1 sola, ahora se multiplicó por 3 (una para Enero, otra para Febrero y otra para Marzo).

# Ejemplo 2: pivot_table() - El Resumen Ejecutivo

El Problema: El dueño de la empresa te dice: "No quiero ver el detalle por meses. Dime cuál es el total de dinero (Suma) que hizo cada Sucursal en cada Categoría".

La Solución: Usar pivot_table() sobre nuestro DataFrame largo para crear una tabla dinámica matemática.

In [ ]:
# Tomamos el df_largo y lo volvemos a cruzar pero sumando los valores
df_resumen = df_largo.pivot_table(
    index = 'sucursal', # Lo que irá en las filas (hacia abajo)
    columns = 'categoria', # Lo que irá en las columnas (hacia la derecha)
    values = 'ventas', # Los números a calcular
    aggfunc = sum , # Sumamos las ventas
    fill_value = 0 # Si alguien no vendió nada, pon un 0 en vez de NaN
)

print(" --- Resumen Total por Sucursal y Categoria ---")
print(df_resumen)

¿Qué pasó aquí?
Pandas buscó todas las filas de "Bogotá" que decían "Laptops" (Enero, Febrero, Marzo), las sumó en el fondo (5000 + 5200 + 4800 = 15000) y te entregó un reporte limpio y cruzado. Esto es exactamente lo que hace Excel con sus tablas dinámicas, pero aquí lo haces con 5 líneas de código que nunca se trabarán aunque tengas millones de filas.

# Solución 3: pivot() - El Cambio de Formato Rápido

In [ ]:
# Para usar pivot simple (sin matemáticas), los datos NO pueden tener duplicados
# Filtramos solo las laptops de Bogotá para el ejemplo
df_bogota_laptops = df_largo[(df_largo['sucursal']=='Bogota')&(df_largo['categoria']=='Laptops')]

# Volvemos a expandir los meses hacia la derecha
df_expandido = df_bogota_laptops.pivot(
    index='categoria',
    columns='mes',
    values='ventas',
)

print("--- De vuelta a formato ancho (Solo Laptops Bogotá) ---")
print(df_expandido)

¿Qué pasó aquí?
Tomamos una tabla vertical y la volvimos a hacer horizontal.
⚠️ Nota Pro: Si hubiéramos intentado hacer un pivot simple de TODA la tabla original, Pandas habría arrojado un error (ValueError) porque no sabría qué hacer con los meses repetidos en diferentes sucursales. ¡Por eso pivot_table es el rey, porque la suma (aggfunc) soluciona ese conflicto!